In [45]:
import GnuGo
import Goban
import random
import gnugoPlayer

gnugo = GnuGo.GnuGo(9)

def query(q):
    ok, r = gnugo.query(q)
    assert ok == "OK"

    return ok, r

def query_score():
    ok, r = query("estimate_score")
    player, score = r.split(" ")[1].split("+")

    if score == "0.0":
        return 0.0

    if player == "W":
        return -float(score)
    else:
        return float(score)

def make_random_evaluations(nb_moves=150):
    evaluations = []
    
    query("clear_board")

    board = Goban.Board()
    moves = 0

    while moves < nb_moves and not board.is_game_over():
        move = random.choice(board.legal_moves())
        query(f"play {board.player_name(board.next_player())} {board.flat_to_name(move)}")
        board.push(move)
        query(f"estimate_score")
        score = query_score()
        evaluations.append((board.get_board().copy(), score))
        moves += 1

    return evaluations

def make_GNUGo_evaluations():
    evaluations = []
    
    query("clear_board")

    board = Goban.Board()
    p1 = gnugoPlayer.myPlayer()
    p1.newGame(1)

    while not board.is_game_over():
        move = None

        if board.next_player() == 1:
            move = p1.getPlayerMove()
        else:
            move = board.flat_to_name(random.choice(board.legal_moves()))
            p1.playOpponentMove(move)

        query(f"play {board.player_name(board.next_player())} {move}")
        board.push(board.name_to_flat(move))

        score = query_score()
        evaluations.append((board.get_board().copy(), score))

    return evaluations

In [46]:
import pickle

dataset = []

try:
    f = open("dataset.pkl", "rb")
    dataset = pickle.load(f)
    f.close()
except:
    print("Cannot load")

for _ in range(500):
    try:
        dataset += make_GNUGo_evaluations()
        dataset += make_random_evaluations()
    
    except:
        print("error yolo")
        break

f = open("dataset.pkl", "wb")
pickle.dump(dataset, f)
f.close()

print("Dataset is " + str(len(dataset)) + " long")

error yolo
Dataset is 14233 long




***assertion failure:
move_reasons.c:348 - next_lunch < 2*19*19/3 near PASS***


   A B C D E F G H J
 9 O O . . O X X X O 9
 8 . . X . . . . . O 8
 7 X X X . . O + . O 7
 6 . X . X . X . O . 6
 5 X O . O X O . . O 5
 4 O X X . O X . . O 4
 3 . O + X X . + . O 3
 2 O . . O X O . . . 2     WHITE (O) has captured 0 stones
 1 O X X X X . O . . 1     BLACK (X) has captured 1 stones
   A B C D E F G H J
(;GM[1]FF[4]SZ[9]KM[0.0]HA[0]GN[GNU Go 3.8 stepped on a bug]
;B[eh];W[fc];B[fd];W[de];B[ff];W[dh];B[bc];W[bg];B[cb];W[hd];B[di];W[if]
;B[dd];W[ia];B[fa];W[ie];B[cc];W[ah];B[];W[aa];B[bi];W[ea];B[bf];W[ad]
;B[ee];W[ig];B[eg];W[ba];B[bd];W[ic];B[ci];W[ef];B[ac];W[be];B[ae];W[ai]
;B[ha];W[gi];B[dg];W[fh];B[cf];W[af];B[ei];W[ib];B[ga];W[fe]
)
gnugo 3.8 (seed 523124675): You stepped on a bug.
Please mail this message, including the debug output above, to gnugo@gnu.org

